### Telegram Scratch

In [80]:
import numpy as np
import pandas as pd
from telethon import TelegramClient
import os
import re
import requests
import json
import asyncio

import requests

In [2]:
base_path = os.path.dirname(os.getcwd())
thr_int_path = os.path.join(base_path, 'Threat_intel')
ref_path = os.path.join(thr_int_path, 'Reference')
output_path = os.path.join(thr_int_path, 'Output')

In [57]:
os.chdir(ref_path)
# import scraping functions
import scrape
tel_tags = pd.read_csv('telegram_tags.csv')
# grab app_id and hash
api_id = int(tel_tags['App_ID'][tel_tags['Source']=='Telegram'].iloc[0])
api_hash = tel_tags['Hash'][tel_tags['Source']=='Telegram'].iloc[0]

In [58]:
client=TelegramClient('anon', api_id, api_hash)

### Sample Functions from Telethon

**<font color=cyan>Send test message to own channel</font>**

In [8]:
async def main():
    async with client: 
        await client.send_message('me','Hello from Python!')
# with client:
#     client.loop.run_until_complete(main())
await main()

In [25]:
#  Join a new channel
from telethon.tl.functions.channels import JoinChannelRequest
async def join_channel(client,channel_link):
    try:
        if not client.is_connected():
            await client.connect()

        await client(JoinChannelRequest(channel_link))
        print(f'Succesfully joined channel {channel_link}')

    except Exception as e:
        print(f'Failed to join channel {channel_link}', e)

async def main():
    async with client:
        channel_link = 'https://t.me/BinancePumpTracker'
        await join_channel(client, channel_link) 

await main()

Succesfully joined channel https://t.me/BinancePumpTracker


### Scrape Messages

In [25]:
num_msgs = 5

async def scrape_message(client, channel, limit=100):
    message_list = []
    try:
        if not client.is_connected():
            await client.connect()  # Ensure connection before making requests
        
        async for each_message in client.iter_messages(channel, limit=limit):
            if each_message.text:  # Check if the message has text content
                message_list.append(each_message.stringify())
                # print(each_message.stringify())  # Print the message content
    except Exception as e: 
        print(f'Failed connection to channel {channel}. Error: {e}')
    return message_list

# async def main():
async with client:  # Ensure the client is properly connected
    # channel_link = 'https://t.me/BinancePumpTracker'
    channel_link = 'BinancePumpTracker'  # Use the channel username, NOT the full URL!
    message_list = await scrape_message(client, channel_link, 5) 

# async def main():
    # async with client:  # Ensure the client is properly connected
    #     # channel_link = 'https://t.me/BinancePumpTracker'
    #     channel_link = 'BinancePumpTracker'  # Use the channel username, NOT the full URL!
    #     message_list = await scrape_message(client, channel_link, 5) 


In [66]:
msg_list_proc = []
# regex_start = r"message='"
regex_start = r'message=(["\'])(.*?)\1' # captures either single or double quote
regex_end = r"out=False"
re_cut = r'\\(n|t|xa0|\\)'

for i in range(num_msgs):
    msg_str = message_list[i]
    msg_str_stage1 = scrape.pullsubstr_begend(regex_start, regex_end, msg_str)
    msg_str_stage2 = re.sub(re_cut, '', msg_str_stage1)
    msg_list_proc.append(msg_str_stage2)

In [69]:
#print sample messages 
for i in range(2):
    print(msg_list_proc[i])

message='#PIVX 👈 https://www.binance.com/en/trade/PIVX_BTCBuying Zone 310-330 Sell🤑 360🤑 360- 400🚀 400- 450 💃 450 -500 & aboveBullish above- 300🔼Death Zone below-300🔽',
	
message='#ADX 👈https://www.binance.com/en/trade/ADX_BTCBuying Zone 230-250 👆Sell🤑 270-280🤑 280-310🚀 310-340💃 340-370 😱 370- 400 & aboveBullish over - 220',
	


**<font color=cyan> Look up quoted tickers and add info from CoinMarketCap </font>**  
Tickers are generally characterized by: 
- '#' or a '$'
- All caps. 

In [84]:
reg_ticker =  r'[#\$][A-Z]+'  # Identifies ticker preceded by 
symbols_list_stage = [list(set(re.findall(reg_ticker, msg))) for msg in msg_list_proc]
symbols_list = [[symbol[1:] for symbol in symbols] for symbols in symbols_list_stage]
symbols_list = [list(set(sym_list)) for sym_list in symbols_list]

**Create Dataframe to store messages, tickers, and later: market info**

In [85]:
df_msg_mkt = pd.DataFrame(msg_list_proc, columns=['Msg_Scraped'])
# Add symbols_list to table
df_msg_mkt['Tickers'] = symbols_list
df_msg_mkt

,Msg_Scraped,Tickers
0,message='#PIVX 👈 https://www.binance.com/en/tr...,[PIVX]
1,message='#ADX 👈https://www.binance.com/en/trad...,[ADX]
2,"message=""#ADX 👈 ✨Call given hereHit 305Targets...",[ADX]
3,message='🔼 🥰😇🥰🥰 hits an Impressive 202.57% APY...,"[HOLD, C, L, B, CET]"
4,"message=""🚀 CET Hits 197.73% APY!CoinEx's empow...","[C, L, B, LKY, BTC, CET]"


### <font color=cyan>Add Market Info</font>
**[API Documentation](https://coinmarketcap.com/api/documentation/v1/#operation/getV1CryptocurrencyListingsHistorical)**

In [64]:
# grab api key
cmc_hash = tel_tags['Hash'][tel_tags['Source']=='Coinmarketcap'].iloc[0]

# set up headers and portal
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': cmc_hash,  # Replace with your API key
}

In [105]:
cmc_vals_list = [] # will house list values from CMC

coin_names_list = []
last_pxs_list = []
mkt_caps_list = []
perc_ch_1h_list = []
perc_ch_1d_list = []
perc_ch_1wk_list = []

for sym_list in symbols_list:
    # convert list into string separated by commas
    sym_list_str = ','.join(sym_list)
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    coin_name_list_stage = []
    last_px_list_stage = []
    mkt_cap_list_stage = []
    perc_ch_1h_list_stage = []
    perc_ch_1d_list_stage = []
    perc_ch_1wk_list_stage = []
    
    for symbol in data['data']:
        try:
            coin = data['data'][symbol]
            coin_name_list_stage.append(coin['name'])
            last_px_list_stage.append(coin['quote']['USD']['price'])
            mkt_cap_list_stage.append(coin['quote']['USD']['market_cap'])
            perc_ch_1h_list_stage.append(coin['quote']['USD']['percent_change_1h'])
            perc_ch_1d_list_stage.append(coin['quote']['USD']['percent_change_24h'])
            perc_ch_1wk_list_stage.append(coin['quote']['USD']['percent_change_7d'])
                                  
        except Exception as e:
            print(f'failed on symbol {symbol}')

    coin_names_list.append(coin_name_list_stage)
    last_pxs_list.append(last_px_list_stage)
    mkt_caps_list.append(mkt_cap_list_stage)
    perc_ch_1h_list.append(perc_ch_1h_list_stage)
    perc_ch_1d_list.append(perc_ch_1d_list_stage)
    perc_ch_1wk_list.append(perc_ch_1wk_list_stage)

In [113]:
df_msg_mkt['Name'] = coin_names_list
df_msg_mkt['Price'] = last_pxs_list
df_msg_mkt['mkt_cap']=mkt_caps_list
df_msg_mkt['Perc_ch_1h'] = perc_ch_1h_list
df_msg_mkt['Perc_ch_1d'] = perc_ch_1d_list
df_msg_mkt['Perc_ch_1wk'] = perc_ch_1wk_list
df_msg_mkt

,Msg_Scraped,Tickers,Name,Price,mkt_cap,Perc_ch_1h,Perc_ch_1d,Perc_ch_1wk
0,message='#PIVX 👈 https://www.binance.com/en/tr...,[PIVX],"[Carbon Coin, CoinEx Token, HoldCoin]","[None, 0.0883255058624474, 0.0005272752437625379]","[None, 62590419.601268105, 0]","[0, -0.05197085, 0.93357981]","[0, 0.0165666, -6.76763176]","[3.59206866, -1.90485507, 303.58414551]"
1,message='#ADX 👈https://www.binance.com/en/trad...,[ADX],"[Carbon Coin, CoinEx Token, HoldCoin]","[None, 0.0883255058624474, 0.0005272752437625379]","[None, 62590419.601268105, 0]","[0, -0.05197085, 0.93357981]","[0, 0.0165666, -6.76763176]","[3.59206866, -1.90485507, 303.58414551]"
2,"message=""#ADX 👈 ✨Call given hereHit 305Targets...",[ADX],"[Carbon Coin, CoinEx Token, HoldCoin]","[None, 0.0883255058624474, 0.0005272752437625379]","[None, 62590419.601268105, 0]","[0, -0.05197085, 0.93357981]","[0, 0.0165666, -6.76763176]","[3.59206866, -1.90485507, 303.58414551]"
3,message='🔼 🥰😇🥰🥰 hits an Impressive 202.57% APY...,"[HOLD, C, L, B, CET]","[Carbon Coin, CoinEx Token, HoldCoin]","[None, 0.0883255058624474, 0.0005272752437625379]","[None, 62590419.601268105, 0]","[0, -0.05197085, 0.93357981]","[0, 0.0165666, -6.76763176]","[3.59206866, -1.90485507, 303.58414551]"
4,"message=""🚀 CET Hits 197.73% APY!CoinEx's empow...","[C, L, B, LKY, BTC, CET]","[Carbon Coin, CoinEx Token, HoldCoin]","[None, 0.0883255058624474, 0.0005272752437625379]","[None, 62590419.601268105, 0]","[0, -0.05197085, 0.93357981]","[0, 0.0165666, -6.76763176]","[3.59206866, -1.90485507, 303.58414551]"
